# Test Program

## Goal

Goal is to use object information from the Blender plugin "write_object_data". This is done in several stages.

## Preliminaries

```console
$ conda install -c conda-forge openexr
```

## Steps implemented and done in this program

1. Load all object information and images
   1. Run through meta data and load it into memory
   1. Load images (including rendered image, depth buffer, classified image
1. Prepare a list of images as background
   1. Define path to images (this will include sub directories)
   1. Run through the directory to load a list of all potential background images
   1. Provide cache mechanism (will be explained below)
1. Implement function to load a random image with random background
   1. Will be implemented as function that uses the cache for the backgorund and merges
1. Run through training process
   1. TBD

In [ ]:
import os
import ntpath
import re

import math

import multiprocessing as mp

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import keras
import operator
#from operator import add
#from operator import 

#import runai.ga

#from nvidia.dali.pipeline import Pipeline
#import nvidia.dali.ops as ops
#import nvidia.dali.types as types

from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Convolution2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.utils.np_utils import to_categorical

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from imgaug import augmenters as iaa

import tensorflow as tf
from tensorflow.python.client import device_lib

import datetime

import cv2
import pandas as pd
import random
from IPython.display import Image
from pathlib import Path

import glob
import json
#import OpenEXR
import imageio

In [ ]:
print( "Number of processors: ", mp.cpu_count() )

In [ ]:
def load_obj_data( jsonfile ):
    json_path = os.path.join( path_to_images, jsonfile )
    with open( json_path, 'r' ) as infile:
        data = json.load( infile )
    return data

In [ ]:
def get_file_name( json_data ):
    img_files_pre = json_data[ "frame" ][ "files" ]
    img_files = [ os.path.normpath( f ) for f in img_files_pre ]
    path_img           = [ f for f in img_files if re.match(r'.*_img[0-9]+\.(png|jpeg|jpg)', f ) ][ 0 ]
    path_img_z_map     = [ f for f in img_files if re.match(r'.*_z[0-9]+\.exr', f ) ][ 0 ]
    path_img_class     = [ f for f in img_files if re.match(r'.*_class[0-9]+\.(png|jpeg|jpg)', f )][ 0 ]
    path_img_z_img     = [ f for f in img_files if re.match(r'.*_z_img[0-9]+\.(png|jpeg|jpg)', f )][ 0 ]
    path_img_class_img = [ f for f in img_files if re.match(r'.*_class_img[0-9]+\.(png|jpeg|jpg)', f )][ 0 ]
    return ( path_img, path_img_z_map, path_img_class, path_img_z_img, path_img_class_img )

In [ ]:
#def img_preprocess_load( img_path ):
#    img = mpimg.imread( img_path )
#    img = img_preprocess( img )
#    return img

In [ ]:
#def img_preprocess( img ):
#    img = img[ 60:135, :, : ]
#    img = cv2.cvtColor( img, cv2.COLOR_RGB2YUV )
#    img = cv2.GaussianBlur( img, ( 3, 3 ), 0 )
#    img = cv2.resize( img, ( 200, 66 ) )
#    img = img / 255
#    return img

In [ ]:
# Helper function to set all alpha values to 1 (not transparent)
# Used to improve visualization in some cases, not required as some part of image processing for the training itself
def img_rm_alpha( img_orig ):
    img = np.copy( img_orig )
    h, w, cp = img.shape
    if cp == 4:
        for y in range( h - 1 ):
            for x in range( w - 1 ):
                img[y][x][3]=1
    return img

In [ ]:
# set to true to get more output for debugging issues
print_debug = False

# path to folder with all json files with object information
path_to_images = "C:\\tmp\\dump_truck4"
#path_to_images = "C:\\tmp\\test"


included_extensions = [ 'json' ] #, 'jpg','jpeg', 'bmp', 'png', 'gif']
obj_data_files = [ fn for fn in os.listdir( path_to_images )
                      if any( fn.endswith( ext ) for ext in included_extensions ) ]
print( "Found ", len( obj_data_files ), " json files" )

In [ ]:
# Section to test the functions "load_obj_data", "get_file_name"

obj_data_idx = random.randint( 0, len( obj_data_files ) - 1 )
obj_data_path = obj_data_files[ obj_data_idx ]
obj_data = load_obj_data(obj_data_path)

obj_data["frame"]["files"]

#####################################################################

path_img, path_img_z_map, path_img_class, path_img_z_img, path_img_class_img = get_file_name(obj_data)
print("path_img           = ", path_img)
print("path_img_z_map     = ", path_img_z_map)
print("path_img_class     = ", path_img_class)
print("path_img_z_map     = ", path_img_z_img)
print("path_img_class_img = ", path_img_class_img)

#pp = "C:\\tmp\\dump_truck4\\minetruck_img0310.jpg"
#my_file = Path(pp)
#print(pp, " is a file ", os.path.isfile(pp) )
#print(pp, " is a file ", os.path.isdir(pp) )
#print(pp, " is a file ", my_file.exists() )

#img             = mpimg.imread( pp )
#img_z_map       = img
img           = mpimg.imread( path_img )
img_z_map     = imageio.imread( path_img_z_map )
img_class     = mpimg.imread( path_img_class )
img_z_img     = mpimg.imread( path_img_z_img )
img_class_img = mpimg.imread( path_img_class_img )

print (img.shape)
print (img_z_map.shape)
print (img_class.shape)
print (img_z_img.shape)
print (img_class_img.shape)
#print (golden[512][640])


fig, axs = plt.subplots( 3, 2, figsize = ( 15, 10 ) )
fig.tight_layout()
axs[ 0 ][ 0 ].set_facecolor("green")
#axs[ 0 ][ 0 ].fill_between([0,1],[1,1], color="none", hatch="X", edgecolor="b", linewidth=0.0)
axs[ 0 ][ 0 ].imshow( img )
axs[ 0 ][ 0 ].grid( color='w', linestyle='-', linewidth = 1 )
axs[ 0 ][ 0 ].set_title( 'Original Image' )

axs[ 1 ][ 0 ].set_facecolor("green")
axs[ 1 ][ 0 ].imshow( img_z_map )
axs[ 1 ][ 0 ].grid( color='w', linestyle='-', linewidth = 1 )
axs[ 1 ][ 0 ].set_title( 'Depth Buffer' )

axs[ 2 ][ 0 ].set_facecolor("green")
axs[ 2 ][ 0 ].imshow( img_class )
axs[ 2 ][ 0 ].grid( color='w', linestyle='-', linewidth = 1 )
axs[ 2 ][ 0 ].set_title( 'Class' )

axs[ 0 ][ 1 ].set_facecolor("green")
axs[ 0 ][ 1 ].imshow( img_z_img )
axs[ 0 ][ 1 ].grid( color='w', linestyle='-', linewidth = 1 )
axs[ 0 ][ 1 ].set_title( 'Depth Image' )

axs[ 1 ][ 1 ].set_facecolor("green")
axs[ 1 ][ 1 ].imshow( img_class_img )
axs[ 1 ][ 1 ].grid( color='w', linestyle='-', linewidth = 1 )
axs[ 1 ][ 1 ].set_title( 'Class 2' )

#show_image("minetruck_img1001.jpg")
#obj_data["frame"][0]["objects"]["object_0000"]

In [ ]:
def img_rect( img_orig, rects ):
    img = img_rm_alpha( img_orig )
    h, w, cp = img.shape
    print ("H,W,CP = ", h, w, cp)
    for rect in rects:
        (p1, p2, color) = rect
        p1s = ( int(p1[0] * w), int(h - p1[1] * h) )
        p2s = ( int(p2[0] * w), int(h - p2[1] * h) )
        cv2.rectangle( img, p1s, p2s, color, 5 )
    alpha = 0.4  # Transparency factor.
    # Following line overlays transparent rectangle over the image
    img = cv2.addWeighted(img, alpha, img_orig, 1 - alpha, 0)
    return img

def rnd_color():
    r = min(random.randint(0, 8) * 32, 255)
    g = min(random.randint(0, 8) * 32, 255)
    b = min(random.randint(0, 8) * 32, 255)
    return (r, g, b)

def get_bb(data, objName):
    c = rnd_color()
    if print_debug:
        print ("Color for ", objName, " is ", c)

    objData = data["frame"]["objects"][objName]
    if "bb2dWithChildren" in objData:
        bb2d = objData["bb2dWithChildren"]
    elif "bb2d" in objData:
        bb2d = objData["bb2d"]
    if bb2d:
        rect = ( ( bb2d["x1"], bb2d["y1"] ), ( bb2d["x2"], bb2d["y2"] ), c )
    else:
        rect = ( ( 0, 0 ), ( 0, 0 ) )
    return rect

In [ ]:
def output_bounding_box(values):
    # output debugging information about the bounding boxes
    if "bb2d" in values:
        print("{}\t{}\tbb2d=({:.2},{:.2},{:.2},{:.2})".format(keys,values["name"],values["bb2d"]["x1"],values["bb2d"]["y1"],values["bb2d"]["x2"],values["bb2d"]["y2"]) )
    if "bb2dWithChildren" in values:
        print("{}\t{}\tbb2dWC=({:.2},{:.2},{:.2},{:.2})".format(keys,values["name"],values["bb2dWithChildren"]["x1"],values["bb2dWithChildren"]["y1"],values["bb2dWithChildren"]["x2"],values["bb2dWithChildren"]["y2"]) )

In [ ]:
# requires an image with alpha channel
# returns the area that has a non-zero alpha values,
# described by ( ( offset_x, offset_y ), ( width, height ) )
def get_nontransparent_area( img ):
    offset_x = 0
    offset_y = 0
    ( size_x, size_y ) = img.shape[ 0 : 2 ]

    line_is_transparent = True
    while offset_x < size_x and line_is_transparent:
        line_is_transparent = ( np.amax( img[ offset_x , ::, 3:4: ] ) == 0 )
        offset_x = offset_x + 1

    line_is_transparent = True
    while offset_y < size_y and line_is_transparent:
        line_is_transparent = ( np.amax( img[ ::, offset_y , 3:4: ] ) == 0 )
        offset_y = offset_y + 1

    size_x = size_x - offset_x
    size_y = size_y - offset_y

    line_is_transparent = True
    while size_x > 0 and line_is_transparent:
        line_is_transparent = ( np.amax( img[ offset_x + size_x - 1 , ::, 3:4: ] ) == 0 )
        size_x = size_x - 1

    line_is_transparent = True
    while size_y > 0 and line_is_transparent:
        line_is_transparent = ( np.amax( img[ ::, offset_y + size_y - 1 , 3:4: ] ) == 0 )
        size_y = size_y - 1

    offset = ( offset_x, offset_y )
    size = ( size_x, size_y )
    return (offset, size)

In [ ]:
print_debug = False

def loadFileInfo( obj_data_path ):
    #objs = [ "Cube.000", "Cube.002", "Cube.003" ]
    #objs = [ "Car Rig", "wheel.Ft.L", "wheel.Ft.R", "wheel.Bk.L", "wheel.Bk.R", "body", "Minetruck" ]
    #objs = [ "wheel.Ft.L", "wheel.Ft.R", "wheel.Bk.L", "wheel.Bk.R", "body", "Minetruck" ]
    #objs = [ "testcar-Body", "testcar-Wheel.Ft.L", "testcar-Wheel.Ft.R", "testcar-Wheel.Bk.L", "testcar-Wheel.Bk.R", "Car Rig" ]
    objs = [ "Car Rig", "wheel.Ft.L", "wheel.Ft.R", "wheel.Bk.L", "wheel.Bk.R" ]

    # load json information
    data = load_obj_data( obj_data_path )

    # load image for this description
    path_img, path_img_z_map, path_img_class, path_img_z_img, path_img_class_img = get_file_name( data )
    #if print_debug:
    print ( "Loading ", obj_data_path, " using image file ", path_img )
    img_original = mpimg.imread( path_img )

    if print_debug:
        # output debugging information about the bounding boxes
        for keys, values in data[ "frame" ][ "objects" ].items():
            output_bounding_box( values )

    # find information about the steering of the vehicle
    for keys, values in data[ "frame" ][ "objects" ][ objs[ 0 ] ][ "pose" ][ "bones" ].items():
        if values[ "name" ] == "Steering":
            steering_bone = values
        elif values[ "name" ] == "GroundSensor.Axle.Ft":
            front_axle_bone = values

    if print_debug and steering_bone:
        print( "Found steering bone: ", steering_bone[ "head" ] )
    if print_debug and front_axle_bone:
        print( "Found front axle bone: ", front_axle_bone[ "head" ] )
    steering_angle = 0.0
    if steering_bone[ "head" ][ "y" ] != 0.0:
        diff_x = steering_bone[ "head" ][ "x" ] - front_axle_bone[ "head" ][ "x" ]
        diff_y = steering_bone[ "head" ][ "y" ] - front_axle_bone[ "head" ][ "y" ]
        steering_angle = 180.0 / math.pi * math.atan( diff_x / diff_y )
    if print_debug:
        print( "steering_angle = ", steering_angle, " degree" )

    nontransparent_area = get_nontransparent_area( img_original )
    rects = [ get_bb( data, o ) for o in objs ]
    return ( steering_angle, rects, img_original, nontransparent_area )

In [ ]:
# run single-threaded
objectInfos = [ loadFileInfo( n ) for n in obj_data_files ]

# run multi-threaded:
#with mp.Pool( processes = mp.cpu_count() ) as pool:
#    objectInfos = pool.map( loadFileInfo, [ n for n in obj_data_files[ :20 ] ] )

In [ ]:
# draw the plots + bounding boxes
#fig, axs = plt.subplots( 1, 1, figsize = ( 15, 30 ) )
#fig.tight_layout()
#obj_data_idx = 948 #random.randint( 0, len( objectInfos ) - 1 )
#( steering_angle, bbs, img ) = loadFileInfo( obj_data_files[ obj_data_idx ] )
#print ( "steering angle = ", steering_angle, " for image ", obj_data_idx )
#axs.imshow( img_rect( img, bbs ) )
#axs.grid( color='w', linestyle='-', linewidth = 1 )

In [ ]:
# draw the plots + bounding boxes
fig, axs = plt.subplots( 4, 2, figsize = ( 15, 30 ) )
fig.tight_layout()
for y in range(4):
    for x in range(2):
        obj_data_idx = random.randint( 0, len( objectInfos ) - 1 )
        ( steering_angle, bbs, img, _ ) = objectInfos[ obj_data_idx ]
        print ( "steering angle = ", steering_angle, " for image ", obj_data_idx )
        axs[ y, x ].imshow( img_rect( img, bbs ) )
        axs[ y, x ].grid( color='w', linestyle='-', linewidth = 1 )
        axs[ y, x ].set_title( 'Original Image (idx = ' + str( obj_data_idx ) + ')' )

In [ ]:
image_extensions = [ 'jpg','jpeg', 'bmp', 'png', 'gif']
background_image_source_folder = "S:\\Bilder_Filme"

def get_sub_dirs( p ):
    subdirs = [ p ]
    for x in p.iterdir():
        if x.is_dir():
            subdirs = subdirs + get_sub_dirs( x )
    return subdirs


bgimg_path = Path( background_image_source_folder )
subdirs = get_sub_dirs( Path( bgimg_path ) )
img_files = []
for path in subdirs:
    print ( "Checking ", path )
    img_files = img_files + [ os.path.join( path, fn ) for fn in os.listdir( path ) if any( fn.endswith( ext ) for ext in image_extensions ) ]

print( "Found ", len( img_files ), " images files" )
print( "Images: ", img_files )

In [ ]:
image_sizes = [ img.shape for (_, _, img, _) in objectInfos ]
image_size = image_sizes[ 0 ]
image_correct_size = [ ( isize == image_size ) for isize in image_sizes ]
image_has_incorrect_size = image_correct_size.count( False )
if image_has_incorrect_size > 0:
    print ( "Some images do not have the same size" )

In [ ]:
# img_files
# * load some images
# * write function to add random background image to a given object image detection
# * testing
# * ... (move forward to AI training)    

In [ ]:
def add_rnd_bckgrnd( ):
    img_idx = random.randint( 0, len( img_files ) - 1 )
    path_bckgrnd_img = img_files[ img_idx ]
    bckgrnd_img = mpimg.imread( path_bckgrnd_img )

    obj_data_idx = random.randint( 0, len( obj_data_files ) - 1 )
    obj_data_path = obj_data_files[ obj_data_idx ]
    obj_data = load_obj_data(obj_data_path)
    path_img, path_img_z_map, path_img_class, path_img_z_img, path_img_class_img = get_file_name(obj_data)

    img           = mpimg.imread( path_img )
    img_z_map     = imageio.imread( path_img_z_map )
    img_class     = mpimg.imread( path_img_class )
    img_z_img     = mpimg.imread( path_img_z_img )
    img_class_img = mpimg.imread( path_img_class_img )

    ( offset, size ) = get_nontransparent_area( img )
    # Assumption: pixel ratio for all images is 1:1
    # Determine maximum scale for each dimension, then select
    # the maximum.
    # A scale < 1 means that the background image is smaller
    #             than the actual image and needs to be enlarged
    # A scale > 1 means that the background image is bigger and
    #             can be scaller from 1 downto scale
    # When the maximum allows downsizing, we choose
    # a random scale to downsize the image. If we need to enlarge
    # the image, we take the original image to not to loose details.
    max_scale_x = bckgrnd_img.shape[ 0 ] / img.shape[ 0 ]
    max_scale_y = bckgrnd_img.shape[ 1 ] / img.shape[ 1 ]
    max_scale = min( max_scale_x, max_scale_y )
    if max_scale > 1:
        scale = random.uniform( 1.0, max_scale )
        scale = math.floor( img.shape[ 0 ] * scale ) / img.shape[ 0 ]
    else:
        scale = max_scale
    crop_size = ( math.floor( img.shape[ 0 ] * scale ), math.floor( img.shape[ 1 ] * scale ) )
    # compute 
    max_offset_crop = [ a - b for ( a , b ) in zip( bckgrnd_img.shape[ 0 : 2 ], crop_size ) ]
    offset_crop = ( random.randint( 0, max_offset_crop[ 0 ] ), random.randint( 0, max_offset_crop[ 1 ] ) )

    ( y, x ) = offset_crop
    ( h, w ) = crop_size
    bckgrnd_crop_img = bckgrnd_img[y:y+h, x:x+w]
    bckgrnd_resize_img = cv2.resize( bckgrnd_crop_img, img.shape[ 1 :  : -1 ] )

    sy, sx, nc = bckgrnd_resize_img.shape
    if nc > 3:
        bckgrnd_resize_img = bckgrnd_resize_img[::,::,0:3:]
    elif nc < 3:
        print( "bckgrnd_resize_img has not enough color channels" )
        pass

    if isinstance( bckgrnd_resize_img[0][0][0], ( float, np.float32, np.float64 ) ):
        bckgrnd_resize_img = ( np.multiply( bckgrnd_resize_img, 255.0 ) ) [ :, :, : ]

    if isinstance( img[0][0][0], ( float, np.float32, np.float64 ) ):
        img_alpha = img[ ::, ::, 3:4: ]
        img_col = ( np.multiply( img, 255.0 ) ) [ ::, ::, 0:3: ]
    elif isinstance( img[0][0][0], ( int, np.uint8 ) ):
        img_alpha = ( np.multiply( img, 1.0 / 255 ) ) [ ::, ::, 3:4: ]
        img_col = img[ ::, ::, 0:3: ]

    sy, sx, _ = img_alpha.shape
    
    # compute mask
    # blend input image and overlay
    img_mixed = cv2.convertScaleAbs(img_col * img_alpha + bckgrnd_resize_img * (1.0 - img_alpha) )

    #return (img_mixed, bckgrnd_crop_img, bckgrnd_resize_img)
    return img_mixed

In [ ]:
# draw the plots + bounding boxes
num_pics_x = 2
num_pics_y = 4
fig, axs = plt.subplots( num_pics_y, num_pics_x, figsize = ( 15, 30 ) )
fig.tight_layout()
for j in range( 0, num_pics_y ):
    for i in range( 0, num_pics_x ):
        img = add_rnd_bckgrnd( )
        if num_pics_x == 1 and num_pics_y == 1:
            p = axs
        elif num_pics_x == 1:
            p = axs[j]
        elif num_pics_y == 1:
            p = axs[i]
        else:
            p = axs[j,i]
        p.imshow( img )
        p.grid( color='w', linestyle='-', linewidth = 1 )
